In [1]:
import requests
import pandas as pd
import numpy as np

#### Definitions ####

In [2]:
owners = ['Luke','Emma','Fat','Olivia','Claire','Matt','George']
owner_ids = ['18900','18935','18967','19180','19424','20043','20626']
team_details_url = {}
lineup_url = {}

for owner_id, owner in enumerate(owners):
    team_details_url[owner] = 'https://draft.premierleague.com/api/entry/'+owner_ids[owner_id]+'/history'
    for gameweek in range(1,39):
        lineup_url[owner,gameweek] = 'https://draft.premierleague.com/api/entry/'+owner_ids[owner_id]+'/event/'+str(gameweek)

bootstrap_url = 'https://draft.premierleague.com/api/bootstrap-static'
player_owner_url = 'https://draft.premierleague.com/api/league/5857/element-status'
league_details_url = 'https://draft.premierleague.com/api/league/5857/details'

current_gameweek = requests.get(bootstrap_url).json()['events']['current']

In [3]:
def url_to_df(url, datatype):
    request = requests.get(url)
    json = request.json()
    df = pd.DataFrame(json[datatype])
    return df

#### Create Dataframes ####

In [18]:
player_types_df = url_to_df(bootstrap_url,'element_types')

teams_df = url_to_df(bootstrap_url,'teams')

player_owner_df = url_to_df(player_owner_url,'element_status')
player_owner_df = player_owner_df.sort_values('element', ignore_index=True)

league_entries_df = url_to_df(league_details_url,'league_entries')
league_standings_df = url_to_df(league_details_url,'standings')
league_standings_df = league_standings_df.sort_values('league_entry', ignore_index=True)
league_standings_df = pd.concat([league_entries_df[['entry_id','id','entry_name','player_first_name','waiver_pick']], league_standings_df[['last_rank','rank','event_total','total']]], axis=1)

player_info_df = url_to_df(bootstrap_url,'elements')
player_info_df['team'] = player_info_df.team.map(teams_df.set_index('id').name)
player_info_df['element_type'] = player_info_df.element_type.map(player_types_df.set_index('id').singular_name)
player_info_df.insert(3,'owner',player_owner_df.owner.map(league_entries_df.set_index('entry_id').player_first_name))

#### Functions for making specific DataFrames ####

In [349]:
def get_lineup_info_by_gameweek(owner, gameweek):
    # check valid gameweek:
    if gameweek > current_gameweek:
        print("Gameweek not yet played")
        return('ERROR')
    
    # create dataframe:
    info_columns = ['name','team','position']
    points_columns = ['minutes','assists','goals_scored','saves','goals_conceded',
                      'own_goals','yellow_cards','red_cards','total_points']
    lineup_df = pd.DataFrame(columns=info_columns + points_columns + ['actual points'],
                             index = url_to_df(lineup_url[owner, gameweek],'picks')['element'])
    
    # all player data:
    lineup_df['name'] = lineup_df.index.map(player_info_df.set_index('id').second_name)
    lineup_df['team'] = lineup_df.index.map(player_info_df.set_index('id').team)
    lineup_df['position'] = lineup_df.index.map(player_info_df.set_index('id').element_type)

    # player by player data:
    for i, player in enumerate(lineup_df.index):
        points_collection = {}
        for data in points_columns:
            points_collection[data] = 0
        player_url = 'https://draft.premierleague.com/api/element-summary/'+str(player)
        player_df = url_to_df(player_url,'history')
        if not player_df.empty:
            player_df = player_df.loc[player_df['event'] == gameweek]
            if not player_df.empty:
                for data in points_columns:
                    points_collection[data] = player_df.sum()[data]
        for data in points_columns:
            lineup_df.loc[lineup_df.index == player, data] = points_collection[data]
        if i < 11:
            lineup_df.loc[lineup_df.index == player, 'actual points'] = lineup_df.loc[lineup_df.index == player, 'total_points']
        else:
            lineup_df.loc[lineup_df.index == player, 'actual points'] = 0
            
    # add totals row:
    totals_df = pd.DataFrame(columns=info_columns + points_columns, index = ['Totals'])
    for data in info_columns:
        totals_df.loc[:, data] = ''
    for data in points_columns + ['actual points']:
        totals_df.loc[:, data] = lineup_df.sum()[data]
    lineup_df = pd.concat([lineup_df, totals_df])
        
    # make reader friendly:
    lineup_df.index.name = None
    lineup_df = lineup_df.rename(columns={'goals_scored': 'goals scored', 'goals_conceded': 'goals conceded',
                                          'own_goals': 'own goals', 'yellow_cards': 'yellow cards',
                                          'red_cards': 'red cards', 'total_points': 'total points'})
    return(lineup_df)

In [423]:
def get_league_standings(Cumulative=True):
    gameweek_range = range(1,current_gameweek+1)
    # create dataframe:
    league_standings_df = pd.DataFrame(columns = owners, index = gameweek_range)
    league_standings_df = league_standings_df.append(pd.Series(name='Totals',dtype=int))
    team_info_df = {}
    for owner in owners:
        team_info_df[owner] = url_to_df(team_details_url[owner],'history')
        for week in gameweek_range:
            points = team_info_df[owner].loc[team_info_df[owner]['event'] == week,'points'].iloc[0]
            league_standings_df.loc[league_standings_df.index==week, owner] = points
        league_standings_df.loc['Totals', owner] = team_info_df[owner].sum()['points']
        
    return(league_standings_df)

In [424]:
get_league_standings(True)

,Luke,Emma,Fat,Olivia,Claire,Matt,George
1,27,43,48,31,39,22,26
Totals,27,43,48,31,39,22,26


In [111]:
team_info_df['Matt'].sum(axis=0)['points']

22.0

In [8]:
slim_players_df = pd.DataFrame(player_info_df[['second_name','first_name','team','owner','element_type','minutes','total_points']])


In [9]:
slim_players_df

,second_name,first_name,team,owner,element_type,minutes,total_points
0,Özil,Mesut,Arsenal,NaN,Midfielder,0,0
1,Papastathopoulos,Sokratis,Arsenal,NaN,Defender,0,0
2,Luiz Moreira Marinho,David,Arsenal,NaN,Defender,0,0
3,Aubameyang,Pierre-Emerick,Arsenal,Olivia,Midfielder,90,7
4,Soares,Cédric,Arsenal,NaN,Defender,0,0
...,...,...,...,...,...,...,...
524,Odoi,Denis,Fulham,NaN,Defender,90,1
525,Forster,Fraser,Southampton,NaN,Goalkeeper,0,0
526,Tella,Nathan,Southampton,NaN,Midfielder,0,0
527,Steele,Jason,Brighton,NaN,Goalkeeper,0,0


In [122]:
slim_players_df.loc[slim_players_df.owner=='Fat']


,second_name,first_name,team,owner,element_type,minutes,total_points
36,Grealish,Jack,Aston Villa,Fat,Midfielder,0,0
73,White,Ben,Brighton,Fat,Defender,78,1
91,Lowton,Matthew,Burnley,Fat,Defender,0,0
119,Mount,Mason,Chelsea,Fat,Midfielder,90,2
154,Digne,Lucas,Everton,Fat,Defender,90,12
249,van Dijk,Virgil,Liverpool,Fat,Defender,90,7
344,Basham,Chris,Sheffield Utd,Fat,Defender,70,1
354,Lundstram,John,Sheffield Utd,Fat,Midfielder,90,2
362,McCarthy,Alex,Southampton,Fat,Goalkeeper,90,2
364,Redmond,Nathan,Southampton,Fat,Midfielder,90,2
